In [1]:
!pip install transformers
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.7 MB/s 
     |████████████████████████████████| 182 kB 86.6 MB/s 
     |████████████████████████████████| 7.6 MB 49.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 28.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
accumulated_grad = 2
epochs = 2
lr_init = 1e-5
batch_size = 4
adam_epsilon = 1e-8
warmup_steps = 3
block_size = 800
max_grad_norm = 1.0
model_save_interval = 2000
val_interval = 100

In [4]:
import re
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#from transformers.tokenization_utils_base import BatchEncoding
import pandas as pd
import numpy as np
import os
from transformers import AdamW, get_linear_schedule_with_warmup
import tensorboardX as tb
from tensorboardX.summary import Summary
#from transformers import GPT2Tokenizer, GPT2LMHeadModel

## Data Cleaning(Skipped)(Ran on local machine)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
df_conversation = pd.read_csv('movie_scripts/movie_conversations.tsv', sep='\t', header=None)
df_movie = pd.read_csv('movie_scripts/movie_titles_metadata.tsv', sep='\t', header=None)

In [ ]:
import csv
df_list = []
with open("movie_scripts/movie_lines.tsv") as f:
   
  # Read data line by line
  for line in f:
     
    # split data by tab
    # store it in list
    l = line.split('\t')
    l1 = l[:4]
    l1.append(' '.join(l[4:]))
    # append list to ans
    df_list.append(l1)
df_line = pd.DataFrame(df_list, columns=['lineID','characterID','movieID','character name','utterance'])

In [ ]:
df_comedy_movie = df_movie.loc[df_movie[5].str.contains('comedy', na=False).values,:]

In [ ]:
df_comedy_conversation = df_conversation.loc[df_conversation[2].isin(df_comedy_movie[0].values),:].reset_index(drop=True)

In [ ]:
df_line['lineID'] = df_line['lineID'].str.replace(r'\"', '', regex=True)

In [ ]:
import re
final_comedy = []
for i in range(df_comedy_conversation.shape[0]):
    if (i+1)% 100 == 0:
        print('Step:',i+1)
    tmp_list = re.sub(r"\'", "", df_comedy_conversation[3][i].strip('[|]')).split()
    base_lineID = tmp_list[0]
    #print(base_lineID)
    base_utter = df_line[df_line.lineID.isin([base_lineID])]['utterance'].item()
    base_tokens = tokenizer.encode(base_utter) + [tokenizer.eos_token_id]
    for j in range(len(tmp_list)-1):
        response_lineID = tmp_list[j+1]
        response_utter = df_line[df_line.lineID.isin([response_lineID])]['utterance'].item()
        response_tokens = tokenizer.encode(response_utter) + [tokenizer.eos_token_id]
        base_tokens = base_tokens + response_tokens
        #final_comedy.append(base_tokens)  
    final_comedy.append(base_tokens)

In [ ]:
df_comedy_final = pd.DataFrame([str(tmp_list) for tmp_list in final_comedy], columns=['utter'])

In [ ]:
#df_comedy_final.to_csv('comedy_tokens.csv', index=False)
df_comedy_final.to_csv('comedy_tokens_simplified.csv', index=False)

## Reading csv

In [5]:
test_length = 300

In [6]:
df_final = pd.read_csv('/content/drive/MyDrive/data/Movie_Dialogpt/comedy_dialogues/comedy_tokens_simplified.csv')

In [7]:
shuffled_df = df_final.sample(frac=1).reset_index(drop=True)

In [8]:
shuffled_df['utter'] = shuffled_df['utter'].str.strip('[|]').str.split(', ')\
.apply(lambda x: [int(input_id) for input_id in x])

In [9]:
df_train = shuffled_df.loc[:len(shuffled_df)-test_length-1]

In [10]:
df_test = shuffled_df.loc[len(shuffled_df)-test_length:].reset_index(drop=True)

## Batch Data

In [11]:
def batch_data(data):
    for epoch in range(epochs):
      for i in range(len(data)):
        X = data.loc[i]["utter"]
        yield X

## Model & Optimizer & Scheduler

In [12]:
resume = False
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

if not resume:
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

    optimizer = AdamW(model.parameters(), lr=lr_init, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs*len(df_train)/batch_size)
else:
    model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/data/Movie_Dialogpt/models/training_args.bin')
     
    optimizer = AdamW(model.parameters(), lr=lr_init, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs*len(df_train)/batch_size)
    
    optimizer.load_state_dict(torch.load("/content/drive/MyDrive/data/Movie_Dialogpt/models/optimizer.pt"))
    scheduler.load_state_dict(torch.load("/content/drive/MyDrive/data/Movie_Dialogpt/models/scheduler.pt"))

if torch.cuda.is_available():
    device = torch.device('cuda:0') # GPU
else:
    device = torch.device('cpu') # CPU
model.to(device)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/351M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

## Training DialoGPT

In [13]:
def evaluate(val_tokens, tb_writer, step):
    for i in range(val_tokens.shape[0]):
        tmp_tokens = val_tokens['utter'][i]
        stop = [i for i,e in enumerate(tmp_tokens) if e==tokenizer.eos_token_id][-2]
        base_tokens = torch.tensor([tmp_tokens[:stop+1]],device=device)
        true_text = tokenizer.decode(tmp_tokens, skip_special_tokens=True)
        predicted_tokens = model.generate(base_tokens, max_length=500)
        predicted_text = tokenizer.decode(predicted_tokens[0], skip_special_tokens=True)
        #print(predicted_tokens)
        print('### STEP: ', step+1)
        print('##### True_Text')
        print('\t'+true_text)
        print('##### Predicted Text')
        print('\t'+predicted_text)
        tb_writer.add_text(tag='val_true_text_'+str(i), text_string=true_text, global_step=step+1)
        tb_writer.add_text(tag='val_predicted_text'+str(i), text_string=predicted_text, global_step=step+1)

In [ ]:
### Tensorboard Summary & Folder Setup
folder_i = 0
while 1:
    if os.path.exists('/content/drive/MyDrive/data/Movie_Dialogpt/train_log/'+str(folder_i)):
        folder_i = folder_i + 1
    else:
        os.mkdir('/content/drive/MyDrive/data/Movie_Dialogpt/train_log/'+str(folder_i))
        break
tb_writer = tb.SummaryWriter('/content/drive/MyDrive/data/Movie_Dialogpt/train_log/'+str(folder_i))

### Training Process
tr_loss = 0.0
train_batches = batch_data(df_train)
for step, batch in tqdm(enumerate(train_batches)):
    #print('##step: ', step)
    inputs, labels = (batch, batch)
    if len(inputs) > 1024:
        print('#### Max Len Exceeded, Step: ', step+1)
        continue
    inputs = torch.tensor(inputs).to(device)
    labels = torch.tensor(inputs).to(device)
    model.train()
    outputs = model(inputs, labels=labels)
    loss = outputs[0]
    loss.backward()
    tb_writer.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=step+1)
    tr_loss += loss.item()
    
    if (step+1) % batch_size == 0:
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    if (step+1) % val_interval == 0:
        val_tokens = df_test.sample(2).reset_index(drop=True)
        evaluate(val_tokens, tb_writer, step)
    if (step+1) % model_save_interval == 0:
        model.save_pretrained('/content/drive/MyDrive/data/Movie_Dialogpt/models/training_args.bin')
        torch.save(optimizer.state_dict(), "/content/drive/MyDrive/data/Movie_Dialogpt/models/optimizer.pt")
        torch.save(scheduler.state_dict(), "/content/drive/MyDrive/data/Movie_Dialogpt/models/scheduler.pt")
tb_writer.close()

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
97it [00:06, 26.96it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architec

### STEP:  100
##### True_Text
	I could never cause you grief; consider our wedding off.  You returned this Westley to his ship?
Yes.
Then we will simply alert him.  Beloved are you certain he still wants you? After all it was you who did the leaving in the Fire Swamp. Not to mention that pirates are not known to be men of their words.

##### Predicted Text
	I could never cause you grief; consider our wedding off.  You returned this Westley to his ship?
Yes.

### STEP:  100
##### True_Text
	I also saw the girl give you this.
Oh yeah that was Mayolia she's a nice girl.  She's kind of strange though.

##### Predicted Text
	I also saw the girl give you this.



199it [00:10, 26.94it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
202it [00:10, 23.39it/s]

### STEP:  200
##### True_Text
	And carrying.  The National Flower of Albania something..... Now Is it some Festival?  Some <u>Harvest</u> Festival something...
Get on this right now...
...got to be <u>something</u>.  She gives him the sacred...  And tells him:  this is the Traditional Harvest Offering given to the Man who Ties the First Sheaf the last sheaf whatever the fuck it is...
...this's <u>good</u>...
Stanley Moss Ladeesngennlmen... And she says...
Is she saying this in Albanian...?
<u>Yes</u>:  she is saying this in Albanian <u>but</u> she's doing it not to be uh...  But because she explains that is the only way it can be understood by her Aged and Sainted Mother who is standing there with her dressed in her Traditional...

##### Predicted Text
	And carrying.  The National Flower of Albania something..... Now Is it some Festival?  Some <u>Harvest</u> Festival something...
Get on this right now...
...got to be <u>something</u>.  She gives him the sacred...  And tells him:  this

298it [00:13, 27.19it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
301it [00:14, 17.94it/s]

...

34699it [26:14, 26.96it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


### STEP:  34700
##### True_Text
	Oh God!
I am.  You-you made my day.
Oh wow!
It was just great.  Uh I was all set...I was set to be bored stiff.
Uh gee.  Would you like to have lunch?  Uh uh...
I-I would love to talk to you about uh that script.  I-I you know I think maybe that we could do something with it.
Okay and listen I would like to hear what made you suddenly decide to drop out of life.
Oh who cares?
Y-you used to-- Oh no!  Yeah I care.  You used to be so ambitious and...  God you really liked it?!

##### Predicted Text
	Oh God!
I am.  You-you made my day.
Oh wow!
It was just great.  Uh I was all set...I was set to be bored stiff.
Uh gee.  Would you like to have lunch?  Uh uh...
I-I would love to talk to you about uh that script.  I-I you know I think maybe that we could do something with it.
Okay and listen I would like to hear what made you suddenly decide to drop out of life.
Oh who cares?
I'm sorry.  I'm sorry.  I'm sorry.  I'm sorry.  I'm sorry.  I'm sorry.  I'm sorry.  I

34798it [26:24, 26.69it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
34801it [26:24, 14.86it/s]

### STEP:  34800
##### True_Text
	Come <i>on</i>.
Shit.
What?
They're gonna rape the maid aren't they.
From the looks of those boys the Ripper too.
And then they'll kill 'em.
At least.

##### Predicted Text
	Come <i>on</i>.
Shit.
What?
They're gonna rape the maid aren't they.
From the looks of those boys the Ripper too.
And then they'll kill 'em.
I'm not saying that.

### STEP:  34800
##### True_Text
	Phyllis I'm sorry. I didn't mean to shout...
No Dr. Delbanco it is I who am sorry. Sorry to have wasted your time with such...

##### Predicted Text
	Phyllis I'm sorry. I didn't mean to shout...
I'm sorry.



34861it [26:26, 26.30it/s]

## References

In [ ]:

# # Let's chat for 5 lines
# for step in range(1):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(user_inputs[step] + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens, 
#     chat_history_ids = model.generate(bot_input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, labels=bot_input_ids)
#     print(chat_history_ids)
#     # pretty print last ouput tokens from bot
#     #print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
# tokenizer.decode(model.generate(bot_input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, labels=bot_input_ids)[0])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'Does money buy happiness?<|endoftext|>Money buys happiness.<|endoftext|>'